# 1. Setup ViZDoom

In [1]:
!pip install vizdoom

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!cd github_clone & git clone https://github.com/Farama-Foundation/ViZDoom

fatal: destination path 'ViZDoom' already exists and is not an empty directory.


In [3]:
# ViZDoom for game env
from vizdoom import *
# Random for action sampling
import random
# Time for sleeping
import time
# Identity matrix
import numpy as np

In [4]:
# Setup game
game = DoomGame()
game.load_config('github_clone/ViZDoom/scenarios/basic.cfg')
game.init()

In [5]:
# Define the set of actions within this Doom env
actions = np.identity(3, dtype=np.uint8)

In [6]:
episodes = 10

for episode in range(episodes):
    # Each episode is a replay of a game
    game.new_episode()
    
    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        # Ammo
        info = state.game_variables
        # Take action, skip next 4 frames, and return the result (better for our agent to see the action take place)
        reward = game.make_action(random.choice(actions), 4)
        print('reward: ', reward)
        time.sleep(0.02)
    print('Result: ', game.get_total_reward())
    time.sleep(2)

reward:  -4.0
reward:  -9.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -9.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -9.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  97.0
Result:  14.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0


ViZDoomUnexpectedExitException: Controlled ViZDoom instance exited unexpectedly.

In [ ]:
game.close()

# 2. Convert to Gym Environment

In [ ]:
!pip install gym

In [8]:
# Env base class
from gym import Env
# Gym spaces
# Discrete: Represents our actions
# Box: An array of any shape
from gym.spaces import Discrete, Box
# OpenCV
import cv2

In [9]:
# ViZDoom env
class VizDoomGym(Env):
    
    # Start the env
    def __init__(self, render=False):
        # Inherit from our Env class
        super().__init__()
        # Setup game
        self.game = DoomGame()
        self.game.load_config('github_clone/ViZDoom/scenarios/basic.cfg')
        
        # Allow us to train the model without having to open a new window each time
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        self.game.init()
        
        # Create spaces
        # game.get_state().screen_buffer.shape
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8)
        self.action_space = Discrete(3)
        
    # Represents a step in the env
    def step(self, action):
        # Specify action and take step
        actions = np.identity(3, dtype=np.uint8)
        reward = self.game.make_action(actions[action], 4)
        
        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            info = self.game.get_state().game_variables
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0
            
        done = self.game.is_episode_finished()
        
        return state, reward, done, info
    
    # Render the env
    def render():
        pass
    
    # New game
    def reset(self):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    # Grayscale the game frame and resize it
    # Reshaping the array into the form that cvtColor expects it in (channel should be at the end)
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation,0,-1), cv2.COLOR_BGR2GRAY)
        # Scales it down so less pixels to process
        # Reducing what does not bring value when come time to train the model
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        
        return state
    
    # Close the env
    def close(self):
        self.game.close()

In [10]:
env = VizDoomGym(render=True)

In [11]:
env.close()

In [ ]:
# Env checker
from stable_baselines3.common import env_checker

In [ ]:
# Make this work by changing one of the things in step function to this
# ammo = self.game.get_state().game_variables[0]
# info = {"ammo: ", ammo}
#env_checker.check_env(env)

In [12]:
!pip install matplotlib
from matplotlib import pyplot as plt
#plt.imshow(state)

Defaulting to user installation because normal site-packages is not writeable


# 3. Setup Callback

In [14]:
!pip install torch torchvision torchaudio
# PPO for RL
!pip install stable-baselines3[extra]

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [15]:
# OS for file
import os
# Callback
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
# Save our model every x number of steps
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [ ]:
CHECKPOINT_DIR = './train/train_basic'
LOG_DIR = './logs/log_basic'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# 4. Train the Model

# 5. Load the Model